In [347]:
%run -n main.py
# cache = set(list_json_cache())
cache = {}
urls = []
for okogu, size in [(OKOGU1, OKOGU1_SIZE), (OKOGU2, OKOGU2_SIZE)]:
    for page in get_pages_count(size, 50, 1):
        url = get_clearspending_select_okogu_url(okogu, page=page, perpage=50)
        if url not in cache:
            urls.append(url)

# fetch_jsons(log_progress(urls, every=1)) 

zakupki_customer_records = list(load_zakupki_customers(urls))

In [367]:
%run -n main.py
cache = set(list_json_cache())
urls = set()
for record in log_progress(zakupki_customer_records, every=10):
    inn = record.inn
    url = get_clearspending_select_inn_url(inn)
    data = load_json(url)
    if data:
        size = data['contracts']['total']
        for page in get_pages_count(size, 50, 1):
            url = get_clearspending_select_inn_url(inn, page=page, perpage=50)
            urls.add(url)
            
# fetch_jsons(log_progress(urls, every=1))

In [380]:
contracts = [
    _
    for _ in load_contracts(log_progress(urls, every=10))
    if _.date and _.date.year == 2015
]

In [435]:
# okpd_names = list(okpd_names())
# dump_contract_products(contracts, okpd_names)

In [441]:
CONTRACT_PRODUCTS_GROUPS = os.path.join(DATA_DIR, 'contract_products_groups.xlsx')
CONTRACT_PRODUCTS_GROUPED = os.path.join(DATA_DIR, 'contract_products_grouped.xlsx')


table = read_excel(CONTRACT_PRODUCTS_GROUPS)
table.columns = ['id', 'name', 'count', 'price', 'group']

view = table.groupby('group').sum()
view = view.sort_values('count', ascending=False)
view.to_excel(CONTRACT_PRODUCTS_GROUPED)

In [468]:
total = 0
for contract in sample(contracts, 10000):
    for product in contract.products:
        if product.okpd and product.okpd.id in ('16.00.11.121', '16.00.11.122', '16.00.11.129'):
            print product.money, '\t', product.name
            total += product.money

207300.0 	Сигареты с фильтром
147900.0 	Сигареты с фильтром
132300.0 	Сигареты с фильтром
115640.0 	Сигареты Максим Red
116164.5 	Сигареты Святой Георгий 9
59500.0 	Сигареты "Максим" красный
26435.5 	Сигареты "Винстон" красный
87450.0 	Сигареты с фильтром "Winston Blue"
79500.0 	Сигареты с фильтром "Winston Classic"
95268.0 	Сигареты с фильтром "More Blue"
164945.0 	Сигареты с фильтром "More Red"
100.46 	Сигареты с фильтром "More Red"
93000.0 	Сигареты с фильтром "Bond Street Red Selection"
55000.0 	Сигареты с фильтром "Балканская звезда классик"
124000.0 	Сигареты с фильтром "Wings Blue"
128000.0 	Сигареты с фильтром "Wings Red"
70000.0 	Сигареты с фильтром "LM Blue"
54500.0 	Сигареты с фильтром "Alliance Classic"


In [76]:
%run -n main.py
# fetch_html(ATLAS_REGIONS)

# region_urls = set(load_atlas_region_urls())

# fetch_htmls(log_progress(region_urls, every=1))

# atlas_urls = {
#     url for region_url in log_progress(region_urls, every=1)
#     for url in load_atlas_urls(region_url)
# }

# fetch_htmls(log_progress(atlas_urls, every=1))

# atlas_records = [
#     load_atlas_page(_)
#     for _ in log_progress(atlas_urls, every=10)
# ]
# dump_atlas_pages(atlas_records)
atlas_records = list(load_atlas_pages())

In [81]:
# cache = set(list_json_cache())
# urls = []
# for record in zakupki_customer_records + atlas_records:
#     address = record.address
#     if address:
#         url = get_geocode_url(address)
#         if url not in cache:
#             urls.append(url)

# fetch_jsons(log_progress(urls, every=1))

# address_coordinates = [
#     Address(_.address, load_coordinates(_.address))
#     for _ in log_progress(zakupki_customer_records + atlas_records, every=100)
#     if _.address
# ]

# dump_zakupki_atlas_join_check(zakupki_customer_records, atlas_records)

inn_atlas_records = list(load_zakupki_atlas_join_check())

In [346]:
# cache = set(list_json_cache())
# urls = {get_bus_search_url(_.inn) for _ in inn_atlas_records} - cache
# fetch_jsons(log_progress(urls, every=1))

bus_search_records = [
    load_bus_search_record(_.inn) for _
    in log_progress(inn_atlas_records, every=100)
]

# cache = set(list_json_cache())
# urls = {get_bus_latest_report_url(_.id) for _ in bus_search_records if _.id} - cache
# fetch_jsons(log_progress(urls, every=1))

bus_report_years = [
    load_bus_report_years(_.id)
    for _ in bus_search_records
    if _.id
]

# cache = set(list_json_cache())
# urls = {
#     get_bus_report_url(_.id, _.year_2014)
#     for _ in bus_report_years if _.year_2014
# } - cache
# fetch_jsons(log_progress(urls, every=1))

bus_reports = [
    load_bus_report(_.id, _.year_2014)
    for _ in bus_report_years if _.year_2014
]

In [345]:
%run -n main.py    
dump_bus_reports(bus_reports, bus_search_records)